In [ ]:
import numpy as np
import torch
import time
import sys
import vgi
import vgi.ct as ct

target_path = 'testdata/012_100_60_sino.npy'
target_sino = np.load(target_path)
sino_shape = target_sino.shape
rec_shape = (256, 256)
target_dir, filename, extname = vgi.parsePath(target_path)

print(target_sino.shape)
vgi.showImage(vgi.normalize(target_sino))

ctg = ct.FanTomoG(rec_shape = rec_shape,
                  sino_shape = sino_shape, 
                  sino = target_sino,  
                  ssim_window = 7, 
                  g_min = None, g_max = None, 
                  scan_range = (0, np.pi * 2), 
                  angles = None, 
                  proj_mode = 'fanflat', 
                  det_width = 1.0, source_origin = 256., origin_det = 256.,
                  algo = 'FBP', iterations = 10, gpu = True)

#n = 20000
#n = 12000
#n = 10000
#n = 3000
n = 1000
shrink_rate = 0.995
shrink_batches = 10
loss = 'ssimL1'
ssimL1_alpha = 0.84
min_decline = 0.000001
random_n = 50
verbose = 1# | 256
log_n = 100

n_reopt = 10
n_div = 1

time_s = time.time()

#g_min_s = 0.5
g_min_s = 1.0
#g_min_s = 1.5
#g_min_s = 2.0
#g_min_s = 3.5
#g_min_s = 5.0
s_g_min_s = str(int(g_min_s * 10))
ctg.setMinSize(sx = g_min_s, sy = g_min_s)
ctg.ssimL1_alpha = ssimL1_alpha
g_rand_min = None
g_rand_min = np.array(ctg.g_min)
g_rand_min[2] = g_min_s
g_rand_min[3] = g_min_s

_g, _proj, _I = ctg.reconstruct(n = n, shrink_rate = shrink_rate, shrink_batches = shrink_batches, 
                          loss = loss,
                          g_rand_min = g_rand_min,
                          min_decline = min_decline,
                          random_n = random_n,
                          verbose = verbose, log_n = log_n)

proj = vgi.toNumpyImage(_proj)
proj = vgi.normalize(proj[0])
vgi.showImage(proj)   

I = vgi.toNumpyImage(_I).squeeze()
print(vgi.metric(I))
I = np.clip(I, a_min = 0.0, a_max = None)
vgi.showImage(I)

verbose_rounds = (n, 50)
for i in range(n_reopt):
    print('reoptimize ', i)
    _g, _proj, _I = ctg.reoptimize(_g = _g, _proj = _proj, _I = _I, loss = loss, 
                                  rounds = 100, min_decline = min_decline, batch_size = 10, n = 0, clamp = True,
                                  verbose = verbose, verbose_rounds = verbose_rounds, log =  None)

    proj = vgi.toNumpyImage(_proj)
    proj = vgi.normalize(proj[0])
    vgi.showImage(proj)  
    I = vgi.toNumpyImage(_I).squeeze()
    print(vgi.metric(I))
    I = np.clip(I, a_min = 0.0, a_max = None)
    vgi.showImage(I)          
    
print('Total time', time.time() - time_s) 
print('n', n)
print('reopt', n_reopt)
print('loss', loss)
print('shrink rate, shrink_batches', shrink_rate, shrink_batches)



In [ ]:
# Evaluation 
import numpy as np
import torch
import time
import sys
sys.path.append('../')
import vgi
import vgi.ct as ct
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse


#gt_path = 'testdata/025_128_gt.npy'
gt_path = 'testdata/012_100_gt.npy'
gt = np.load(gt_path)
print('gt', gt.shape, vgi.metric(gt))
vgi.showImage(gt)         

I = vgi.toNumpyImage(_I).squeeze()
print(I.shape, vgi.metric(I))
I = np.clip(I, a_min = 0.0, a_max = None)
vgi.showImage(I)
    
diff = I - gt
v_mae = np.mean(np.abs(diff))
v_mse = mse(I, gt) 
v_ssim = ssim(I, gt, data_range = gt.max() - gt.min())
v_psnr = psnr(I, gt, data_range = gt.max() - gt.min())
print('mae', 'mse', 'ssim', 'psnr')
print(v_mae, v_mse, v_ssim, v_psnr)


In [ ]:
# Saving 
import numpy as np
import torch
import time
import sys
sys.path.append('../')
import vgi
import vgi.ct as ct

output_name =  filename + '_ctg_n' + str(n) + '_g' + s_g_min_s
output_path = 'testdata/' + output_name + '.npy'
output_img_path = 'testdata/' + output_name + '.tiff'
        
np.save(output_path, I)
print('npy saved:', output_path)
vgi.saveImage(output_img_path, I)
print('Image saved:', output_img_path)